# TODO

- улучшить парсер (убрать пустые тексты)
- убрать наны из текстов!!!!
- умножить tf-idf каждого слова на каждый вектор в w2v в тексте
- добавить метрики длинв текстов и макс и мин тф-идф


## Метрики
длина текста, max min tf-idf, max и min косинусное расстояние между парами слов (соседних и не соседних, если успеем посчитать), tf-idf взвешенное (умножать каждое слово на tf-idf этого слова для этого документа), слово с самым высоким tf-idf

In [353]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import matplotlib
%matplotlib inline
import random
import warnings

df = pd.read_csv('data.csv', index_col='id')
df = df.sample(frac=1)
df.head()

,text,group
id,,
2126,Голова пришельца с солярным символом | Мексика...,ufo
1553,Да уж.,telegony
785,"Харе Кришна, дорогие преданные! \nХотим подели...",krishnaits
397,Все уже знают про этот замечательный фильм Джо...,podkoren
2615,КУДАХ НЕ ПОЧИТАЕТ СТАРОСТЬ\n\nстарость не радо...,kudah


In [354]:
import warnings
warnings.filterwarnings('ignore')

In [355]:
df.describe(include=[np.object])

,text,group
count,2447,3000
unique,2429,5
top,Доброе утро!,krishnaits
freq,5,600


In [356]:
import nltk
import re
from pymystem3 import Mystem

nltk.download('punkt')
mystem_analyzer = Mystem()

lemmatized = []
unlemmatized = []

for doc in df['text']:
    text = re.sub("<br>", "", str(doc))
    word_list = nltk.word_tokenize(text)
    unlemmatized.append(word_list)
    lemmatized.append([mystem_analyzer.lemmatize(word)[0] for word in word_list])

[nltk_data] Downloading package punkt to /home/valeria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [357]:
from itertools import chain

count_unlem = 0

for el in unlemmatized:
    count_unlem += len(el)

cont_lemmatized = list(chain.from_iterable(lemmatized))

print("Количество токенов в датасете:", count_unlem)
print("Количество уникальных лемм:", len(set(cont_lemmatized)))

Количество токенов в датасете: 692543
Количество уникальных лемм: 37248


In [358]:
words_per_doc = np.mean([len(text) for text in unlemmatized])
symbols_per_doc = np.mean([len(str(doc)) for doc in df["text"]])

print("Среднее количество слов на документ:", int(words_per_doc))
print("Среднее количество символов на документ:", int(symbols_per_doc))

Среднее количество слов на документ: 230
Среднее количество символов на документ: 1281


In [359]:
from nltk.corpus import stopwords
import pymorphy2 as pm2

pmm = pm2.MorphAnalyzer()
russian_stopwords = stopwords.words("russian")

# сюда добавить парсинг получше
def parse(text):
    text = re.sub(r'[^\w\s]','', str(text))
    text = re.sub("[0-9A-Za-z#_=]", "", text)
    text = [pmm.normal_forms(x)[0] for x in text.split() if x not in russian_stopwords]
    if text == [np.nan]:
        return(['nan'])
    return text

df['text'] = df['text'].apply(parse)
df["text"]

2019-06-08 01:15:17,856 : INFO : Loading dictionaries from /usr/local/lib/python3.6/dist-packages/pymorphy2_dicts/data
2019-06-08 01:15:17,990 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


id
2126    [голова, пришелец, солярный, символ, мексика, ...
1553                                                 [да]
785     [хара, кришна, дорогой, предать, хотеть, подел...
397     [весь, знать, замечательный, фильм, джон, карп...
2615    [куд, не, почитать, старость, старость, радост...
2199                   [снимка, плутон, усиленный, цвета]
2649    [куд, на, весь, случай, жизнь, куд, решить, вы...
2477    [куд, непрескоствавенный, день, пожарка, кудах...
2626    [куд, кончить, куд, увидеть, чердак, читать, у...
943     [самадх, гаудия, вайшнав, картик, парикрам, за...
1517                                                   []
369     [племя, внутренний, часть, новый, гвинея, науч...
1198                                                   []
866     [пыль, врадж, исполнять, весь, желание, картик...
2283    [криминалист, подчеркнуть, сомнение, причина, ...
1756    [когда, начинать, жить, делать, нравиться, ост...
2084                             [предполагать, метеорит]
2839    [ку

In [360]:
df

,text,group
id,,
2126,"[голова, пришелец, солярный, символ, мексика, ...",ufo
1553,[да],telegony
785,"[хара, кришна, дорогой, предать, хотеть, подел...",krishnaits
397,"[весь, знать, замечательный, фильм, джон, карп...",podkoren
2615,"[куд, не, почитать, старость, старость, радост...",kudah
2199,"[снимка, плутон, усиленный, цвета]",ufo
2649,"[куд, на, весь, случай, жизнь, куд, решить, вы...",kudah
2477,"[куд, непрескоствавенный, день, пожарка, кудах...",kudah
2626,"[куд, кончить, куд, увидеть, чердак, читать, у...",kudah


In [361]:
df = df.reindex(index=range(len(df)))
cross_val_folds = []

fold = []

for idx, row in df.iterrows():
    fold.append(row)
    if idx % (len(df) // 10) == 0:
        cross_val_folds.append(fold)
        fold = []

len(cross_val_folds)

10

In [362]:
cross_val_folds = cross_val_folds[1:]

for fold in cross_val_folds:
    print(len(fold))

300
300
300
300
300
300
300
300
300


## TF-IDF

In [363]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess

dct = Dictionary()
#for text in lemmatized_corpus:
dct.add_documents(list(df["text"]))
corpus = []
for idx, row in df.iterrows():
    corpus.append(dct.doc2bow(row['text']))
tf_idf = TfidfModel(corpus)

2019-06-08 01:16:10,745 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-08 01:16:11,082 : INFO : built Dictionary(39768 unique tokens: ['антропология', 'большой', 'братец', 'вдохнуть', 'весь']...) from 3000 documents (total 384903 corpus positions)
2019-06-08 01:16:11,655 : INFO : collecting document frequencies
2019-06-08 01:16:11,655 : INFO : PROGRESS: processing document #0
2019-06-08 01:16:11,712 : INFO : calculating IDF weights for 3000 documents and 39768 features (267742 matrix non-zeros)


In [364]:
vectors = [tf_idf[doc] for doc in corpus]

In [365]:
vectors = [{dct[el[0]]: el[1] for el in vector} for vector in vectors]
vectors[0]

{'антропология': 0.13193552536887726,
 'большой': 0.05344479063789524,
 'братец': 0.15032721936852053,
 'вдохнуть': 0.1552589573836702,
 'весь': 0.09518281651471602,
 'вечность': 0.12109074706973376,
 'воскликнуть': 0.12866292143201075,
 'вряд': 0.0921973472432415,
 'вывод': 0.08963642679724876,
 'выполнить': 0.09274880429267256,
 'выпустить': 0.10163798396933127,
 'глубоко': 0.09776731505494081,
 'говориться': 0.08528866133929136,
 'грань': 0.11552077925803443,
 'должный': 0.047119136143471956,
 'дорогой': 0.07444388304014787,
 'ещё': 0.03700306878934491,
 'закончиться': 0.09923202700494284,
 'знать': 0.042512466224095914,
 'и': 0.030564587722126505,
 'иметь': 0.15479264270676557,
 'исследование': 0.09574388920952373,
 'история': 0.05175899247409029,
 'итог': 0.15131470121113363,
 'конец': 0.05233391859077387,
 'корень': 0.16205688399197954,
 'который': 0.057524361579211584,
 'кто': 0.05415846801491459,
 'культурный': 0.08451657979140888,
 'литература': 0.07831455195453832,
 'любезный

## Embeddings

In [88]:
!wget "http://corpus.leeds.ac.uk/tools/ru/ruwac-parsed.out.xz"

--2019-06-06 23:14:42--  http://corpus.leeds.ac.uk/tools/ru/ruwac-parsed.out.xz
Распознаётся corpus.leeds.ac.uk (corpus.leeds.ac.uk)… 129.11.80.147
Подключение к corpus.leeds.ac.uk (corpus.leeds.ac.uk)|129.11.80.147|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 9768295304 (9,1G) [application/x-xz]
Сохранение в: «ruwac-parsed.out.xz.1»

ruwac-parsed.out.xz   0%[                    ]   6,39M  1,41MB/s    ост 2h 8m  ^C


In [ ]:
import lzma
import os
from collections import Counter
from datetime import datetime
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LsiModel
from progressbar import progressbar
from sklearn import svm
from sklearn.exceptions import UndefinedMetricWarning
from sklearn import metrics 
from sklearn.model_selection import LeaveOneOut, cross_val_score, GridSearchCV, train_test_split
from string import punctuation as punct
from string import ascii_letters as latin
from tqdm import tqdm
punct += '»«'

corpus_archive_filename = 'ruwac-parsed.out.xz'
corpus_filename = 'corpus.txt'
premodel_filename = 'small.model'
vocabulary_filename = 'freq.txt'
model_filename = 'full.model'
sample_directory = 'sample'
manual_filename = 'manual_metrics.csv'
df_filename = 'data.csv'
lsi_model_small_filename = 'small_lsi.model'
wordlist_filename = 'wordlist.json'
idfs_filename = 'idfs.json'


with lzma.open(corpus_archive_filename, mode='rt', encoding='utf-8') as f:
    with open(corpus_filename, mode='w', encoding='utf-8') as t:
        current = []
        for line_number, line in enumerate(f):
            if line_number % 10000000 == 0:
                print(line_number, datetime.now().time(), file=open('archive_log.txt', 'a', encoding='utf-8'))
            l = line.split('\t')
            if len(l) == 7:
                lemm = l[3]
                sent = l[1]
                if sent == 'SENT':
                    if set(latin).intersection(set(''.join(current))) == set():
                        current.append('\n')
                        print(' '.join(current), file=t, end='')
                    current = []
                elif lemm not in punct:
                    current.append(lemm)

In [81]:
from gensim.models import Word2Vec
import logging

In [82]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [83]:
sentences = list(df["text"])
sentences[:2]

[['любезный',
  'братец',
  'сестричка',
  'под',
  'корень',
  'закончиться',
  'весь',
  'иметь',
  'начать',
  'должный',
  'иметь',
  'конец',
  'но',
  'весь',
  'мы',
  'нужный',
  'вывод',
  'последний',
  'сладкий',
  'итог',
  'часть',
  'итог',
  'любезный',
  'кто',
  'знать',
  'такой',
  'пойти',
  'прорасти',
  'но',
  'отпустить',
  'пустой',
  'рука',
  'неправильно',
  'поэтому',
  'написать',
  'большой',
  'трактат',
  'под',
  'корень',
  'это',
  'свой',
  'род',
  'исследование',
  'выполнить',
  'грань',
  'социология',
  'литература',
  'история',
  'культурный',
  'антропология',
  'трактат',
  'описывать',
  'тот',
  'потаённый',
  'позиция',
  'который',
  'иметь',
  'смысл',
  'смотреть',
  'вечность',
  'современный',
  'мир',
  'трактат',
  'являться',
  'простой',
  'сумма',
  'предыдущий',
  'текст',
  'он',
  'написать',
  'нуль',
  'учёт',
  'что',
  'говориться',
  'наверное',
  'такой',
  'вряд',
  'ещё',
  'прочитать',
  'это',
  'тот',
  'накопитьс

In [84]:
premodel = Word2Vec(sentences, min_count = 1)

2019-06-06 20:20:46,585 : INFO : collecting all words and their counts
2019-06-06 20:20:46,588 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-06 20:20:46,722 : INFO : collected 39768 word types from a corpus of 384903 raw words and 3000 sentences
2019-06-06 20:20:46,722 : INFO : Loading a fresh vocabulary
2019-06-06 20:20:46,981 : INFO : effective_min_count=1 retains 39768 unique words (100% of original 39768, drops 0)
2019-06-06 20:20:46,982 : INFO : effective_min_count=1 leaves 384903 word corpus (100% of original 384903, drops 0)
2019-06-06 20:20:47,124 : INFO : deleting the raw counts dictionary of 39768 items
2019-06-06 20:20:47,127 : INFO : sample=0.001 downsamples 21 most-common words
2019-06-06 20:20:47,128 : INFO : downsampling leaves estimated 371555 word corpus (96.5% of prior 384903)
2019-06-06 20:20:47,269 : INFO : estimated required memory for 39768 words and 100 dimensions: 51698400 bytes
2019-06-06 20:20:47,269 : INFO : resetting laye

In [85]:
list(premodel.wv.vocab.items())[:10]

[('любезный', <gensim.models.keyedvectors.Vocab at 0x7ff1c6fa07f0>),
 ('братец', <gensim.models.keyedvectors.Vocab at 0x7ff1cf0b4940>),
 ('сестричка', <gensim.models.keyedvectors.Vocab at 0x7ff1cf0b4ac8>),
 ('под', <gensim.models.keyedvectors.Vocab at 0x7ff1cf0b4a90>),
 ('корень', <gensim.models.keyedvectors.Vocab at 0x7ff1cf0b48d0>),
 ('закончиться', <gensim.models.keyedvectors.Vocab at 0x7ff1cf0b4908>),
 ('весь', <gensim.models.keyedvectors.Vocab at 0x7ff1cf0b4d68>),
 ('иметь', <gensim.models.keyedvectors.Vocab at 0x7ff1cf0b4f60>),
 ('начать', <gensim.models.keyedvectors.Vocab at 0x7ff1cf0b4978>),
 ('должный', <gensim.models.keyedvectors.Vocab at 0x7ff1cf0b4710>)]

In [86]:
premodel.similarity('сестричка', 'убить')

0.9995198

In [147]:
addition = "\n"

for sent in sentences:
    addition += " ".join(sent) + '\n'
    
with open(corpus_filename, mode='a', encoding='utf-8') as t:
    t.write(addition)

In [148]:
def get_vocabulary(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            for word in line.split():
                yield word

def get_sentences(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            yield line.split()

In [149]:
from collections import Counter

word_freq = Counter(tqdm(get_vocabulary(corpus_filename)))



0it [00:00, ?it/s]

127330it [00:00, 1273295.14it/s]

287403it [00:00, 1356540.17it/s]

470748it [00:00, 1471356.77it/s]

653198it [00:00, 1562059.19it/s]

840061it [00:00, 1642920.76it/s]

1031568it [00:00, 1716080.37it/s]

1214028it [00:00, 1747254.32it/s]

1396246it [00:00, 1769076.23it/s]

1590530it [00:00, 1817851.02it/s]

1768896it [00:01, 1762118.73it/s]

1943087it [00:01, 1696358.22it/s]

2111835it [00:01, 1481374.56it/s]

2264257it [00:01, 1212633.53it/s]

2396513it [00:01, 1114875.56it/s]

2517243it [00:01, 1130011.73it/s]

2636764it [00:01, 1064643.75it/s]

2748569it [00:01, 1032236.50it/s]

2871762it [00:02, 1084998.30it/s]

3019247it [00:02, 1178448.18it/s]

3187970it [00:02, 1295659.03it/s]

3370528it [00:02, 1419249.88it/s]

3542207it [00:02, 1497088.31it/s]

3727236it [00:02, 1588021.60it/s]

3892438it [00:02, 1602421.05it/s]

4061504it [00:02, 1627909.55it/s]

4227551it [00:02, 1450452.47it/s]

4378474it [00:03, 1335939.44it/s]

4537183it [00:03, 1402521.96it/s]

468

30636597it [00:21, 1891570.88it/s]

30833169it [00:21, 1913220.72it/s]

31028592it [00:21, 1925340.83it/s]

31222652it [00:21, 1929891.13it/s]

31415774it [00:21, 1907601.18it/s]

31606686it [00:22, 1796609.01it/s]

31787780it [00:22, 1487234.48it/s]

31946991it [00:22, 1517215.48it/s]

32130254it [00:22, 1599817.49it/s]

32297235it [00:22, 1620189.49it/s]

32463616it [00:22, 1480574.69it/s]

32617065it [00:22, 1459296.76it/s]

32800640it [00:22, 1554953.05it/s]

32960553it [00:23, 1468731.15it/s]

33111506it [00:23, 1327543.10it/s]

33277191it [00:23, 1411717.49it/s]

33423635it [00:23, 1376873.64it/s]

33565202it [00:23, 1316653.40it/s]

33700141it [00:23, 1287461.18it/s]

33831323it [00:23, 1255898.75it/s]

33958789it [00:23, 1159337.13it/s]

34100867it [00:23, 1227070.27it/s]

34237875it [00:24, 1266726.53it/s]

34366983it [00:24, 1113057.08it/s]

34483384it [00:24, 1051032.45it/s]

34634852it [00:24, 1157308.68it/s]

34761074it [00:24, 1186902.00it/s]

34884347it [00:24, 1162954.0

63381587it [00:42, 1247323.81it/s]

63559673it [00:42, 1370501.93it/s]

63746343it [00:42, 1489257.12it/s]

63933233it [00:42, 1585903.30it/s]

64125233it [00:42, 1673250.77it/s]

64305457it [00:43, 1709963.27it/s]

64501274it [00:43, 1777554.24it/s]

64697836it [00:43, 1830082.90it/s]

64891139it [00:43, 1859794.94it/s]

65081678it [00:43, 1873241.88it/s]

65274873it [00:43, 1890475.43it/s]

65465219it [00:43, 1853814.37it/s]

65651668it [00:43, 1770393.07it/s]

65830247it [00:43, 1756761.56it/s]

66011411it [00:43, 1772870.80it/s]

66189505it [00:44, 1757909.03it/s]

66365890it [00:44, 1758431.60it/s]

66558555it [00:44, 1805727.87it/s]

66739913it [00:44, 1808074.30it/s]

66923009it [00:44, 1814878.94it/s]

67114652it [00:44, 1844193.93it/s]

67299400it [00:44, 1764504.52it/s]

67476810it [00:44, 1738931.27it/s]

67656379it [00:44, 1755577.80it/s]

67832504it [00:45, 1715465.07it/s]

68004651it [00:45, 1684859.61it/s]

68178406it [00:45, 1700320.25it/s]

68368579it [00:45, 1756115.9

97901295it [01:02, 1357583.31it/s]

98039809it [01:03, 1301952.39it/s]

98172419it [01:03, 1262239.44it/s]

98323379it [01:03, 1327493.65it/s]

98458392it [01:03, 1295791.48it/s]

98589723it [01:03, 1222796.56it/s]

98736888it [01:03, 1288142.91it/s]

98868085it [01:03, 1217416.10it/s]

98992298it [01:03, 1178138.54it/s]

99139903it [01:03, 1254067.11it/s]

99268057it [01:03, 1222734.60it/s]

99392427it [01:04, 1169167.06it/s]

99542569it [01:04, 1252304.30it/s]

99670828it [01:04, 1223156.80it/s]

99795414it [01:04, 1168637.81it/s]

99944722it [01:04, 1250131.75it/s]

100072791it [01:04, 1201203.05it/s]

100195460it [01:04, 1183160.06it/s]

100338045it [01:04, 1246824.43it/s]

100464933it [01:04, 1213247.61it/s]

100588001it [01:05, 1154103.20it/s]

100717691it [01:05, 1193523.26it/s]

100838662it [01:05, 1185314.90it/s]

100958340it [01:05, 1105005.65it/s]

101086096it [01:05, 1151670.53it/s]

101205658it [01:05, 1164511.03it/s]

101323431it [01:05, 1125248.75it/s]

101449476it [01:0

126477459it [01:23, 1423113.54it/s]

126651213it [01:23, 1504526.65it/s]

126828226it [01:23, 1575443.42it/s]

126998286it [01:23, 1611008.49it/s]

127177951it [01:24, 1662543.27it/s]

127346325it [01:24, 1574721.27it/s]

127545579it [01:24, 1680430.40it/s]

127741231it [01:24, 1754713.40it/s]

127920131it [01:24, 1588047.54it/s]

128084359it [01:24, 1392899.68it/s]

128238944it [01:24, 1435508.05it/s]

128388524it [01:24, 1253907.40it/s]

128522274it [01:25, 1219968.83it/s]

128656092it [01:25, 1253178.22it/s]

128785873it [01:25, 1191555.47it/s]

128908787it [01:25, 1196191.31it/s]

129031041it [01:25, 1160453.64it/s]

129159124it [01:25, 1194121.34it/s]

129280235it [01:25, 1142332.31it/s]

129422554it [01:25, 1214215.88it/s]

129578007it [01:25, 1299563.21it/s]

129736458it [01:26, 1373663.57it/s]

129894552it [01:26, 1429902.57it/s]

130052161it [01:26, 1470829.24it/s]

130216847it [01:26, 1519552.66it/s]

130376733it [01:26, 1542504.57it/s]

130532538it [01:26, 1511633.16it/s]

1

...

348424021it [03:42, 1852078.93it/s]

348610979it [03:42, 1857292.64it/s]

348796768it [03:42, 1849336.62it/s]

348981751it [03:42, 1832418.44it/s]

349165063it [03:42, 1818564.35it/s]

349350799it [03:42, 1830025.87it/s]

349540059it [03:42, 1848358.52it/s]

349729596it [03:42, 1862214.47it/s]

349917595it [03:42, 1867510.94it/s]

350104412it [03:42, 1841282.40it/s]

350288670it [03:43, 1766114.20it/s]

350466026it [03:43, 1759820.30it/s]

350642535it [03:43, 1748455.63it/s]

350817766it [03:43, 1719163.75it/s]

350995146it [03:43, 1735193.55it/s]

351178598it [03:43, 1763842.76it/s]

351356036it [03:43, 1766990.02it/s]

351542566it [03:43, 1795376.70it/s]

351722375it [03:43, 1775831.93it/s]

351904599it [03:43, 1789406.71it/s]

352088961it [03:44, 1805330.91it/s]

352276740it [03:44, 1826473.84it/s]

352459565it [03:44, 1824673.59it/s]

352647923it [03:44, 1841952.93it/s]

352836146it [03:44, 1853853.10it/s]

353028837it [03:44, 1875063.56it/s]

353220333it [03:44, 1886846.73it/s]

3

382691799it [04:01, 1703193.26it/s]

382871509it [04:01, 1730317.67it/s]

383056335it [04:01, 1764088.30it/s]

383238327it [04:01, 1780462.82it/s]

383416683it [04:01, 1778324.58it/s]

383594733it [04:01, 1748049.89it/s]

383769806it [04:02, 1732383.80it/s]

383943264it [04:02, 1702388.80it/s]

384114023it [04:02, 1703942.67it/s]

384284607it [04:02, 1678728.53it/s]

384468554it [04:02, 1723917.67it/s]

384656280it [04:02, 1767223.19it/s]

384841496it [04:02, 1791872.88it/s]

385030618it [04:02, 1820562.77it/s]

385223012it [04:02, 1850388.31it/s]

385411575it [04:02, 1860820.93it/s]

385603670it [04:03, 1878226.03it/s]

385791757it [04:03, 1868192.77it/s]

385979885it [04:03, 1872097.65it/s]

386167235it [04:03, 1826354.34it/s]

386350215it [04:03, 1765655.05it/s]

386527459it [04:03, 1765882.99it/s]

386704522it [04:03, 1754491.30it/s]

386880321it [04:03, 1741845.82it/s]

387054771it [04:03, 1736349.35it/s]

387228650it [04:03, 1737078.84it/s]

387411881it [04:04, 1764591.60it/s]

3

415876036it [04:20, 1559752.07it/s]

416048370it [04:20, 1605470.15it/s]

416218096it [04:21, 1631891.79it/s]

416381796it [04:21, 1570688.63it/s]

416539715it [04:21, 1452875.22it/s]

416687353it [04:21, 1458523.62it/s]

416859571it [04:21, 1528735.38it/s]

417040946it [04:21, 1604367.44it/s]

417214946it [04:21, 1642777.60it/s]

417389138it [04:21, 1671312.92it/s]

417569407it [04:21, 1708669.75it/s]

417748733it [04:22, 1733196.13it/s]

417922999it [04:22, 1720831.77it/s]

418097171it [04:22, 1727043.48it/s]

418281672it [04:22, 1760816.60it/s]

418464908it [04:22, 1781685.32it/s]

418647980it [04:22, 1796116.93it/s]

418839069it [04:22, 1829021.29it/s]

419022338it [04:22, 1817299.13it/s]

419204342it [04:22, 1758884.03it/s]

419382141it [04:22, 1764570.56it/s]

419561608it [04:23, 1773493.09it/s]

419739265it [04:23, 1749682.73it/s]

419914521it [04:23, 1557435.72it/s]

420074389it [04:23, 1546109.53it/s]

420249757it [04:23, 1603030.68it/s]

420431655it [04:23, 1662232.11it/s]

4

449814430it [04:40, 1804638.48it/s]

449999723it [04:40, 1818858.78it/s]

450181742it [04:40, 1800928.82it/s]

450361969it [04:40, 1788875.60it/s]

450548007it [04:40, 1809742.29it/s]

450736838it [04:40, 1832615.40it/s]

450926636it [04:40, 1851745.05it/s]

451111974it [04:41, 1846438.93it/s]

451300290it [04:41, 1857300.59it/s]

451490641it [04:41, 1870925.32it/s]

451677826it [04:41, 1867268.59it/s]

451864619it [04:41, 1821045.69it/s]

452047023it [04:41, 1807376.87it/s]

452227994it [04:41, 1792302.39it/s]

452415522it [04:41, 1816413.51it/s]

452602915it [04:41, 1833293.52it/s]

452789084it [04:41, 1841719.00it/s]

452973396it [04:42, 1836100.41it/s]

453160194it [04:42, 1845548.66it/s]

453347715it [04:42, 1854345.97it/s]

453533218it [04:42, 1824107.73it/s]

453715787it [04:42, 1795854.36it/s]

453895582it [04:42, 1770754.00it/s]

454075065it [04:42, 1777906.69it/s]

454253021it [04:42, 1764193.59it/s]

454429580it [04:42, 1750311.26it/s]

454604734it [04:43, 1690291.75it/s]

4

483501794it [04:59, 1630491.38it/s]

483665003it [04:59, 1620545.16it/s]

483827183it [05:00, 1602819.30it/s]

483988850it [05:00, 1606946.56it/s]

484173879it [05:00, 1672952.11it/s]

484341941it [05:00, 1659380.42it/s]

484520460it [05:00, 1695214.70it/s]

484700163it [05:00, 1724525.05it/s]

484883713it [05:00, 1756381.77it/s]

485070077it [05:00, 1787132.74it/s]

485256184it [05:00, 1808687.61it/s]

485437460it [05:00, 1804956.97it/s]

485618243it [05:01, 1787463.10it/s]

485797229it [05:01, 1719100.47it/s]

485969867it [05:01, 1626081.12it/s]

486134028it [05:01, 1565469.87it/s]

486292147it [05:01, 1561456.30it/s]

486457236it [05:01, 1587249.34it/s]

486624869it [05:01, 1612962.67it/s]

486787664it [05:01, 1617426.83it/s]

486954463it [05:01, 1632267.97it/s]

487118074it [05:02, 1578958.56it/s]

487263005it [05:02, 1612821.91it/s]

In [150]:
model = Word2Vec(sg=1, min_count=10, workers=1, size=300)

logging.basicConfig(filename='vocabulary.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
model.build_vocab_from_freq(word_freq)
model.save("new_model")

In [152]:
logging.basicConfig(filename='training.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [153]:
model.train(get_sentences(corpus_filename), epochs=1, total_examples=len(word_freq))

2019-06-07 02:43:45,388 : INFO : training model with 1 workers on 369372 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2019-06-07 02:43:46,405 : INFO : EPOCH 1 - PROGRESS: at 0.20% examples, 65175 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:43:47,436 : INFO : EPOCH 1 - PROGRESS: at 0.48% examples, 72476 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:43:48,446 : INFO : EPOCH 1 - PROGRESS: at 0.75% examples, 72795 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:43:49,535 : INFO : EPOCH 1 - PROGRESS: at 1.03% examples, 75488 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:43:50,590 : INFO : EPOCH 1 - PROGRESS: at 1.35% examples, 77513 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:43:51,652 : INFO : EPOCH 1 - PROGRESS: at 1.63% examples, 78745 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:43:52,703 : INFO : EPOCH 1 - PROGRESS: at 1.92% examples, 79742 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:43:53,780 : INFO : EPOCH 1 - PROGRESS: at 2.19% examples, 

2019-06-07 02:45:03,841 : INFO : EPOCH 1 - PROGRESS: at 20.78% examples, 83762 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:45:04,922 : INFO : EPOCH 1 - PROGRESS: at 21.05% examples, 83764 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:45:05,992 : INFO : EPOCH 1 - PROGRESS: at 21.33% examples, 83779 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:45:07,070 : INFO : EPOCH 1 - PROGRESS: at 21.62% examples, 83789 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:45:08,146 : INFO : EPOCH 1 - PROGRESS: at 21.91% examples, 83800 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:45:09,243 : INFO : EPOCH 1 - PROGRESS: at 22.17% examples, 83789 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:45:10,316 : INFO : EPOCH 1 - PROGRESS: at 22.47% examples, 83799 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:45:11,360 : INFO : EPOCH 1 - PROGRESS: at 22.79% examples, 83836 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:45:12,410 : INFO : EPOCH 1 - PROGRESS: at 23.10% examples, 83860 words/s, in_qsize 1, out_

2019-06-07 02:46:22,690 : INFO : EPOCH 1 - PROGRESS: at 41.48% examples, 83373 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:46:23,742 : INFO : EPOCH 1 - PROGRESS: at 41.77% examples, 83392 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:46:24,826 : INFO : EPOCH 1 - PROGRESS: at 42.07% examples, 83395 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:46:25,834 : INFO : EPOCH 1 - PROGRESS: at 42.32% examples, 83386 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:46:26,896 : INFO : EPOCH 1 - PROGRESS: at 42.62% examples, 83399 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:46:27,915 : INFO : EPOCH 1 - PROGRESS: at 42.87% examples, 83389 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:46:28,993 : INFO : EPOCH 1 - PROGRESS: at 43.16% examples, 83391 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:46:30,072 : INFO : EPOCH 1 - PROGRESS: at 43.47% examples, 83394 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:46:31,136 : INFO : EPOCH 1 - PROGRESS: at 43.75% examples, 83406 words/s, in_qsize 1, out_

2019-06-07 02:47:41,627 : INFO : EPOCH 1 - PROGRESS: at 62.66% examples, 83869 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:47:42,685 : INFO : EPOCH 1 - PROGRESS: at 62.96% examples, 83877 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:47:43,741 : INFO : EPOCH 1 - PROGRESS: at 63.25% examples, 83887 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:47:44,796 : INFO : EPOCH 1 - PROGRESS: at 63.53% examples, 83896 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:47:45,882 : INFO : EPOCH 1 - PROGRESS: at 63.81% examples, 83896 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:47:46,958 : INFO : EPOCH 1 - PROGRESS: at 64.08% examples, 83898 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:47:48,036 : INFO : EPOCH 1 - PROGRESS: at 64.38% examples, 83901 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:47:49,118 : INFO : EPOCH 1 - PROGRESS: at 64.65% examples, 83901 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:47:50,200 : INFO : EPOCH 1 - PROGRESS: at 64.93% examples, 83901 words/s, in_qsize 1, out_

2019-06-07 02:49:00,168 : INFO : EPOCH 1 - PROGRESS: at 84.76% examples, 84128 words/s, in_qsize 2, out_qsize 0
2019-06-07 02:49:01,235 : INFO : EPOCH 1 - PROGRESS: at 85.05% examples, 84132 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:49:02,311 : INFO : EPOCH 1 - PROGRESS: at 85.33% examples, 84132 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:49:03,357 : INFO : EPOCH 1 - PROGRESS: at 85.66% examples, 84139 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:49:04,417 : INFO : EPOCH 1 - PROGRESS: at 85.97% examples, 84145 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:49:05,482 : INFO : EPOCH 1 - PROGRESS: at 86.28% examples, 84149 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:49:06,565 : INFO : EPOCH 1 - PROGRESS: at 86.55% examples, 84149 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:49:07,644 : INFO : EPOCH 1 - PROGRESS: at 86.83% examples, 84147 words/s, in_qsize 1, out_qsize 0
2019-06-07 02:49:08,712 : INFO : EPOCH 1 - PROGRESS: at 87.13% examples, 84150 words/s, in_qsize 1, out_

...

2019-06-07 04:00:45,016 : INFO : EPOCH 1 - PROGRESS: at 1140.02% examples, 83485 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:00:46,039 : INFO : EPOCH 1 - PROGRESS: at 1140.24% examples, 83485 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:00:47,083 : INFO : EPOCH 1 - PROGRESS: at 1140.46% examples, 83484 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:00:48,117 : INFO : EPOCH 1 - PROGRESS: at 1140.67% examples, 83484 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:00:49,174 : INFO : EPOCH 1 - PROGRESS: at 1140.86% examples, 83483 words/s, in_qsize 2, out_qsize 0
2019-06-07 04:00:50,201 : INFO : EPOCH 1 - PROGRESS: at 1141.10% examples, 83483 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:00:51,240 : INFO : EPOCH 1 - PROGRESS: at 1141.31% examples, 83483 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:00:52,281 : INFO : EPOCH 1 - PROGRESS: at 1141.53% examples, 83482 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:00:53,312 : INFO : EPOCH 1 - PROGRESS: at 1141.73% examples, 83482 words/s

2019-06-07 04:02:00,944 : INFO : EPOCH 1 - PROGRESS: at 1158.33% examples, 83488 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:02:02,004 : INFO : EPOCH 1 - PROGRESS: at 1158.61% examples, 83488 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:02:03,073 : INFO : EPOCH 1 - PROGRESS: at 1158.89% examples, 83489 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:02:04,134 : INFO : EPOCH 1 - PROGRESS: at 1159.18% examples, 83489 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:02:05,204 : INFO : EPOCH 1 - PROGRESS: at 1159.46% examples, 83489 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:02:06,263 : INFO : EPOCH 1 - PROGRESS: at 1159.75% examples, 83490 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:02:07,325 : INFO : EPOCH 1 - PROGRESS: at 1160.04% examples, 83490 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:02:08,379 : INFO : EPOCH 1 - PROGRESS: at 1160.34% examples, 83491 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:02:09,444 : INFO : EPOCH 1 - PROGRESS: at 1160.62% examples, 83491 words/s

2019-06-07 04:03:17,215 : INFO : EPOCH 1 - PROGRESS: at 1179.25% examples, 83524 words/s, in_qsize 2, out_qsize 0
2019-06-07 04:03:18,284 : INFO : EPOCH 1 - PROGRESS: at 1179.53% examples, 83525 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:03:19,324 : INFO : EPOCH 1 - PROGRESS: at 1179.83% examples, 83525 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:03:20,351 : INFO : EPOCH 1 - PROGRESS: at 1180.20% examples, 83526 words/s, in_qsize 2, out_qsize 0
2019-06-07 04:03:21,424 : INFO : EPOCH 1 - PROGRESS: at 1180.48% examples, 83527 words/s, in_qsize 2, out_qsize 0
2019-06-07 04:03:22,477 : INFO : EPOCH 1 - PROGRESS: at 1180.82% examples, 83528 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:03:23,486 : INFO : EPOCH 1 - PROGRESS: at 1181.06% examples, 83527 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:03:24,496 : INFO : EPOCH 1 - PROGRESS: at 1181.31% examples, 83527 words/s, in_qsize 2, out_qsize 0
2019-06-07 04:03:25,504 : INFO : EPOCH 1 - PROGRESS: at 1181.57% examples, 83527 words/s

2019-06-07 04:04:33,649 : INFO : EPOCH 1 - PROGRESS: at 1199.76% examples, 83546 words/s, in_qsize 2, out_qsize 0
2019-06-07 04:04:34,721 : INFO : EPOCH 1 - PROGRESS: at 1200.03% examples, 83547 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:04:35,783 : INFO : EPOCH 1 - PROGRESS: at 1200.34% examples, 83547 words/s, in_qsize 2, out_qsize 0
2019-06-07 04:04:36,843 : INFO : EPOCH 1 - PROGRESS: at 1200.62% examples, 83548 words/s, in_qsize 2, out_qsize 0
2019-06-07 04:04:37,899 : INFO : EPOCH 1 - PROGRESS: at 1200.91% examples, 83548 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:04:38,965 : INFO : EPOCH 1 - PROGRESS: at 1201.19% examples, 83548 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:04:40,022 : INFO : EPOCH 1 - PROGRESS: at 1201.48% examples, 83549 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:04:41,092 : INFO : EPOCH 1 - PROGRESS: at 1201.77% examples, 83549 words/s, in_qsize 1, out_qsize 0
2019-06-07 04:04:42,158 : INFO : EPOCH 1 - PROGRESS: at 1202.04% examples, 83550 words/s

(407494680, 487263005)

In [154]:
model.save("new_model")

2019-06-07 04:05:03,344 : INFO : saving Word2Vec object under new_model, separately None
2019-06-07 04:05:03,345 : INFO : storing np array 'vectors' to new_model.wv.vectors.npy
2019-06-07 04:05:04,673 : INFO : not storing attribute vectors_norm
2019-06-07 04:05:04,675 : INFO : storing np array 'syn1neg' to new_model.trainables.syn1neg.npy
2019-06-07 04:05:06,066 : INFO : not storing attribute cum_table
2019-06-07 04:05:06,068 : WARNING : this function is deprecated, use smart_open.open instead
2019-06-07 04:05:06,854 : INFO : saved new_model


In [155]:
list(model.wv.vocab.items())[:10]

[('это', <gensim.models.keyedvectors.Vocab at 0x7ff196901ef0>),
 ('тип', <gensim.models.keyedvectors.Vocab at 0x7ff1e223e9b0>),
 ('девушка', <gensim.models.keyedvectors.Vocab at 0x7ff1e223e748>),
 ('в', <gensim.models.keyedvectors.Vocab at 0x7ff1e223eb00>),
 ('один', <gensim.models.keyedvectors.Vocab at 0x7ff1cc5446a0>),
 ('день', <gensim.models.keyedvectors.Vocab at 0x7ff1cc544630>),
 ('отмечать', <gensim.models.keyedvectors.Vocab at 0x7ff1cc544518>),
 ('что', <gensim.models.keyedvectors.Vocab at 0x7ff1cc544550>),
 ('на', <gensim.models.keyedvectors.Vocab at 0x7ff1cc544748>),
 ('себя', <gensim.models.keyedvectors.Vocab at 0x7ff1cc544780>)]

In [156]:
model.similarity('сестричка', 'но')

0.64816344

In [302]:
model.wv["антропология"]

array([ 1.10730127e-01,  1.26508608e-01, -1.03569642e-01, -3.25532526e-01,
       -2.40161583e-01,  2.19315112e-01,  6.93487227e-02, -1.34364828e-01,
        9.93252173e-02,  2.59713948e-01,  4.82175164e-02, -3.69920701e-01,
        2.51802713e-01, -4.59939465e-02, -2.53417492e-01, -7.20142275e-02,
       -1.06924534e-01, -8.85697901e-02, -1.61396086e-01, -2.93577798e-02,
       -1.33057311e-02, -1.23486720e-01, -2.26309329e-01, -1.59146130e-01,
       -5.18877394e-02,  2.70509701e-02,  1.55852109e-01, -1.33963168e-01,
        8.99688751e-02, -2.35866904e-01,  1.53675333e-01, -5.68326749e-02,
       -5.91897964e-03,  1.31843388e-01, -9.04412102e-03,  5.04717417e-03,
        9.59967300e-02, -2.71536350e-01, -7.74048641e-02,  1.37029454e-01,
       -2.47025415e-01, -1.47608519e-01,  3.38691413e-01,  1.69657141e-01,
       -4.77884784e-02,  1.51492637e-02,  8.71634297e-03, -3.47720355e-01,
        3.44226420e-01,  2.01828003e-01, -1.12429634e-01,  4.55528945e-02,
       -2.11331863e-02,  

In [368]:
df['weighted_vec'] = pd.Series(np.nan, index=df.index)
df.head()

,text,group,weighted_vec
id,,,
0,"[любезный, братец, сестричка, под, корень, зак...",podkoren,NaN
1,"[завтра, воскресение, сообщество, появиться, п...",podkoren,NaN
2,"[за, время, существование, наш, сообщество, со...",podkoren,NaN
3,"[жизнь, продолжаться, продолжаться, дорога, на...",podkoren,NaN
4,"[на, день, погибнуть, наш, автор, любимый, бра...",podkoren,NaN


In [381]:
defect_words = 0

def get_embs(word):
    global defect_words
    try:
        word_emb = model.wv[re.sub("ё", "е", str(word))]
    except KeyError:
        try:
            word_emb = premodel.wv[re.sub("ё", "е", str(word))]
        except KeyError:
            word_emb = 0
            defect_words += 1
    return word_emb

def doc_to_vec(doc, vector):
    vec = [get_embs(word) * vector[word] for word in doc]
    return vec

In [385]:
d2v = [doc_to_vec(df["text"][doc], vectors[doc]) for doc in range(len(df["weighted_vec"]))]
df['weighted_vec'] = pd.Series(d2v, index=df.index)

In [386]:
df.head()

,text,group,weighted_vec
id,,,
0,"[любезный, братец, сестричка, под, корень, зак...",podkoren,"[[0.049963534, 0.030964537, -0.053651962, -0.0..."
1,"[завтра, воскресение, сообщество, появиться, п...",podkoren,"[[0.0043625357, 0.0046413695, 0.0063758204, -0..."
2,"[за, время, существование, наш, сообщество, со...",podkoren,"[[0.014121662, -0.009278245, 0.016219541, -0.0..."
3,"[жизнь, продолжаться, продолжаться, дорога, на...",podkoren,"[[0.013573785, -0.0042355275, 0.008554284, -0...."
4,"[на, день, погибнуть, наш, автор, любимый, бра...",podkoren,"[[0.0005447091, 0.007055029, 0.0078044445, -0...."


In [389]:
df['word_length'] = pd.Series([len(doc) for doc in df["text"]], index=df.index)
df.head()

,text,group,weighted_vec,word_length
id,,,,
0,"[любезный, братец, сестричка, под, корень, зак...",podkoren,"[[0.049963534, 0.030964537, -0.053651962, -0.0...",107
1,"[завтра, воскресение, сообщество, появиться, п...",podkoren,"[[0.0043625357, 0.0046413695, 0.0063758204, -0...",161
2,"[за, время, существование, наш, сообщество, со...",podkoren,"[[0.014121662, -0.009278245, 0.016219541, -0.0...",118
3,"[жизнь, продолжаться, продолжаться, дорога, на...",podkoren,"[[0.013573785, -0.0042355275, 0.008554284, -0....",240
4,"[на, день, погибнуть, наш, автор, любимый, бра...",podkoren,"[[0.0005447091, 0.007055029, 0.0078044445, -0....",233


In [390]:
def symbol_length(doc):
    length = 0
    for word in doc:
        length += len(word)
    return length

df['symbol_length'] = pd.Series([symbol_length(doc) for doc in df["text"]], index=df.index)
df.head()

,text,group,weighted_vec,word_length,symbol_length
id,,,,,
0,"[любезный, братец, сестричка, под, корень, зак...",podkoren,"[[0.049963534, 0.030964537, -0.053651962, -0.0...",107,701
1,"[завтра, воскресение, сообщество, появиться, п...",podkoren,"[[0.0043625357, 0.0046413695, 0.0063758204, -0...",161,1062
2,"[за, время, существование, наш, сообщество, со...",podkoren,"[[0.014121662, -0.009278245, 0.016219541, -0.0...",118,829
3,"[жизнь, продолжаться, продолжаться, дорога, на...",podkoren,"[[0.013573785, -0.0042355275, 0.008554284, -0....",240,1712
4,"[на, день, погибнуть, наш, автор, любимый, бра...",podkoren,"[[0.0005447091, 0.007055029, 0.0078044445, -0....",233,1519


In [392]:
def max_idf(vec):
    ret = 0
    for key in vec:
        if vec[key] > ret:
            ret = vec_key
    return ret

def min_idf(vec):
    ret = 0
    for key in vec:
        if vec[key] < ret:
            ret = vec_key
    return ret

df['max_idf'] = pd.Series([max_idf(vec) for vec in vectors], index=df.index)
df['min_idf'] = pd.Series([min_idf(vec) for vec in vectors], index=df.index)

df.head()

NameError: name 'vec_key' is not defined